# Mintpy Time Series
George Brencher

In [ ]:
# Install MintPy and PyAPS
!conda install -c conda-forge mintpy
!conda install -c conda-forge pyaps3

In [ ]:
# Import required packages
%matplotlib inline
import os
import numpy as np
import matplotlib.pyplot as plt
import mintpy
from mintpy.objects import ifgramStack
from mintpy.utils import plot as pp, utils as ut
from mintpy import view, plot_network
from mintpy.unwrap_error_phase_closure import plot_num_triplet_with_nonzero_integer_ambiguity

In [ ]:
# Function to write to config file
def write_config_file(out_file, CONFIG_TXT, mode='a'): 
    """Write configuration files for MintPy to process ISCE products"""
    if not os.path.isfile(out_file) or mode == 'w':
        with open(out_file, "w") as fid:
            fid.write(CONFIG_TXT)
        print('write configuration to file: {}'.format(out_file))
    else:
        with open(out_file, "a") as fid:
            fid.write("\n" + CONFIG_TXT)
        print('add the following to file: \n{}'.format(CONFIG_TXT))

In [ ]:
# Create required directory
os.chdir('/tmp')
os.makedirs('mintpy')
os.chdir('/tmp/mintpy')

In [ ]:
# View MintPy options
!smallbaselineApp.py --help

In [ ]:
# Write config file for MintPy. For a list of processing options, see: 
#https://nbviewer.org/github/insarlab/MintPy-tutorial/blob/main/smallbaselineApp.ipynb
CONFIG_TXT = '''# vim: set filetype=cfg:
##----------------------------- SentinelStack/ISCE ---------------------##
processor                       = isce
sentinelStack.demDir            = /tmp/DEM
sentinelStack.boundingBox       = '27 28 86.3 87'
sentinelStack.subswath          = 1 2  # comment 
sentinelStack.numConnections    = 2   # comment
sentinelStack.azimuthLooks      = 2   # comment
sentinelStack.rangeLooks        = 9  # comment
sentinelStack.filtStrength      = 0.2  # comment
sentinelStack.unwMethod         = snaphu  # comment
sentinelStack.coregistration    = auto  # comment

##-------------------------------- MintPy -----------------------------##
mintpy.load.processor        = isce
mintpy.load.updateMode       = auto  #[yes / no], auto for yes, skip re-loading if HDF5 files are complete
mintpy.load.compression      = auto  #[gzip / lzf / no], auto for no.
##---------for ISCE only:
mintpy.load.metaFile         = /tmp/stack/IW*.xml
mintpy.load.baselineDir      = /tmp/baselines
##---------interferogram datasets:
mintpy.load.unwFile          = /tmp/merged/interferograms/*/geo_filt_*.unw
mintpy.load.corFile          = /tmp/merged/interferograms/*/geo_filt_*.cor
mintpy.load.connCompFile     = /tmp/merged/interferograms/*/geo_filt_*.unw.conncomp
##---------geometry datasets:
mintpy.load.demFile          = /tmp/merged/geom_reference/hgt.rdr
mintpy.load.lookupYFile      = /tmp/merged/geom_master/lat.rdr
mintpy.load.lookupXFile      = /tmp/merged/geom_reference/lon.rdr
mintpy.load.incAngleFile     = /tmp/merged/geom_reference/los.rdr
mintpy.load.azAngleFile      = /tmp/merged/geom_reference/los.rdr
mintpy.load.shadowMaskFile   = /tmp/merged/geom_reference/shadowMask.rdr
mintpy.load.waterMaskFile    = None

mintpy.reference.lalo        = 27.905810, 86.911164 #update with reference point in radar coordinates
mintpy.subset.lalo = 27.86:28.00,86.80:87.00    #[31.5:32.5,130.5:131.0 / no], auto for no
'''

config_file = os.path.join('/tmp/mintpy', "imja_august2021.txt")
write_config_file(config_file, CONFIG_TXT, mode='w')

In [ ]:
# Load data into MintPy
!smallbaselineApp.py imja_august2021.txt --dostep load_data

In [ ]:
# Examine loaded inputs to MintPy
!ls -l inputs

In [ ]:
# Plot all data related to one interferometric pair
view.main('./inputs/ifgramStack.h5 20210806_20210818 --ncols 3'.split())

In [ ]:
# Examine radar geometry files
view.main('./inputs/geometryRadar.h5'.split())

In [ ]:
# Optional step prior to inversion. Remove interferograms based on coherence, temporal/perpendicular baselines, date, etc. 
!smallbaselineApp.py imja_august2021.txt --dostep modify_network

In [ ]:
# Examine interferogram network 
plot_network.main(['./inputs/ifgramStack.h5'])

In [ ]:
# Select reference point. Should be coherent and close to the aoi. 
!smallbaselineApp.py imja_august2021.txt --dostep reference_point

In [ ]:
# Without inversion, quickly assesses possible deformation and phase unwrapping error
!smallbaselineApp.py imja_august2021.txt --dostep quick_overview

In [ ]:
# Plot results of phase stacking 
view.main('avgPhaseVelocity.h5'.split())

In [ ]:
# Plot number of triplets with nonzero integer ambiguity, related to unwrapping error
plot_num_triplet_with_nonzero_integer_ambiguity('numTriNonzeroIntAmbiguity.h5', display=True, fig_size=[12, 4])

In [ ]:
# Do time series inversion. 
!smallbaselineApp.py imja_august2021.txt --dostep invert_network

In [ ]:
# Plot the raw phase time-series re-wrapped into (-5, 5) cm
view.main('timeseries.h5 --wrap --wrap-range -5 5 -u cm --notitle --notick --noaxis'.split())

In [ ]:
# Plot time series temporal coherence and mask coherence 
view.main('temporalCoherence.h5 -c gray --notick --noaxis --noverbose'.split())
view.main('maskTempCoh.h5 -c gray --notick --noaxis --noverbose'.split())

In [ ]:
# Correct for tropospheric correction. Copernicus Climate Data Store (CDS) registration is needed to access ERA5 data. 
# To create account and set up api, follow steps at https://github.com/insarlab/pyaps#2-account-setup-for-era5
!smallbaselineApp.py imja_august2021.txt --dostep correct_troposphere

In [ ]:
# View tropospheric corrected time series
view.main('timeseries_ERA5.h5 --wrap --wrap-range -5 5 --notitle --notick --noaxis'.split())

In [ ]:
# Remove linear or quadratic ramps based on the phase of reliable pixels. Recommended for localized deformation signals
!smallbaselineApp.py imja_august2021.txt --dostep deramp

In [ ]:
# View deramped time series
view.main('timeseries_ERA5_ramp.h5 --wrap --wrap-range -5 5 --notitle --notick --noaxis'.split())

In [ ]:
# Correct phase residual caused by DEM error
!smallbaselineApp.py imja_august2021.txt --dostep correct_topography

In [ ]:
# View DEM error-corrected time series
view.main('timeseries_ERA5_ramp_demErr.h5 --wrap --wrap-range -5 5 --notitle --notick --noaxis'.split())

In [ ]:
# Calculate average velocity of time series 
!smallbaselineApp.py imja_august2021.txt --dostep velocity

In [ ]:
# View original and troposphere corrected velocities
view.main('velocity.h5 --notick --noaxis --noverbose'.split())
view.main('velocityERA5.h5 --notick --noaxis --noverbose'.split())

In [ ]:
# Geocode outputs
!smallbaselineApp.py imja_august2021.txt --dostep geocode